In [3]:
import pandas as pd

In [2]:
df = pd.read_csv('csv/stock_prices.csv')

In [4]:
df['Symbol'].value_counts()

ACB     17
FIBK    17
FCF     17
CNO     17
SHEN    17
        ..
NEWR     5
TRHC     5
PLM      4
FREQ     3
ESTE     3
Name: Symbol, Length: 2447, dtype: int64

In [4]:
# Specify the path to your TXT file
file_path = 'extra.txt'

# Initialize an empty list to store symbols
symbols = []

# Open the file and read its content
with open(file_path, 'r') as file:
    # Read lines from the file
    lines = file.readlines()

    # Iterate through each line
    for line in lines:
        # Split each line at ':', and get the symbol before ':'
        symbol = line.split(':')[0].strip()

        # Append the symbol to the list if it's not empty
        if symbol:
            symbols.append(symbol)

# Print the list of symbols
print("List of Symbols:", symbols[:10])

List of Symbols: ['FIT', 'HEXO', 'FB', 'ZNGA', 'TWTR', 'APHA', 'VSLR', 'WORK', 'ATVI', 'GLUU']


###  Filter symbol csv

In [7]:
sym_df = pd.read_csv('csv/stock_symbols.csv')
print(sym_df)
# Drop rows where the 'symbol' column is in the list
df_filtered = sym_df[~sym_df['symbol'].isin(symbols)]

# Print the modified DataFrame
print("Filtered DataFrame:")
print(df_filtered) 

df_filtered.to_csv('csv/stock_symbols.csv', index=False)

     symbol
0       ACB
1         F
2        GE
3      MSFT
4      GPRO
...     ...
3130   AIRT
3131    RBC
3132   ROLL
3133   ATRI
3134   HHHH

[3135 rows x 1 columns]
Filtered DataFrame:
     symbol
0       ACB
1         F
2        GE
3      MSFT
4      GPRO
...     ...
3123      B
3127    AIN
3130   AIRT
3131    RBC
3133   ATRI

[2447 rows x 1 columns]


### Filter stock_info csv

In [8]:
info_df = pd.read_csv('csv/stock_info.csv')
print(info_df)
# Drop rows where the 'symbol' column is in the list
df_filtered = info_df[~info_df['symbol'].isin(symbols)]

# Print the modified DataFrame
print("Filtered DataFrame:")
print(df_filtered) 

df_filtered.to_csv('csv/stock_info.csv', index=False)

                                                  name symbol   type  \
0                   Aurora Cannabis Inc. Common Shares    ACB  stock   
1                                   Ford Motor Company      F  stock   
2                             General Electric Company     GE  stock   
3                   Microsoft Corporation Common Stock   MSFT  stock   
4                     GoPro, Inc. Class A Common Stock   GPRO  stock   
...                                                ...    ...    ...   
3130                          Air T, Inc. Common Stock   AIRT  stock   
3131                          Regal Beloit Corporation    RBC  stock   
3132            RBC Bearings Incorporated Common Stock   ROLL  stock   
3133                   Atrion Corporation Common Stock   ATRI  stock   
3134  Wealthbridge Acquisition Limited Ordinary Shares   HHHH  stock   

                                            description  \
0     Aurora Cannabis, Inc. engages in the productio...   
1     Ford Motor 

## Transform

In [12]:
import pandas as pd

# Read the CSV file into a DataFrame
stock_df = pd.read_csv('csv/stock_prices.csv')
print(len(stock_df))
stock_df.head()

79901


,date,Symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2023-10-02 00:00:00-04:00,ACB,0.584,0.584,0.553,0.560,12773300,0.0,0.0,NaN
1,2023-10-03 00:00:00-04:00,ACB,0.555,0.557,0.521,0.537,8659400,0.0,0.0,NaN
2,2023-10-04 00:00:00-04:00,ACB,0.560,0.576,0.535,0.553,9605000,0.0,0.0,NaN
3,2023-10-05 00:00:00-04:00,ACB,0.556,0.580,0.553,0.567,6164100,0.0,0.0,NaN
4,2023-10-06 00:00:00-04:00,ACB,0.552,0.578,0.542,0.567,4979100,0.0,0.0,NaN


In [41]:
#select attributes
clean_df = stock_df[['date', 'Symbol', 'Open','High','Low','Close','Volume','Dividends']]

In [42]:
# Convert the 'Date' column to a datetime object
clean_df['date'] = pd.to_datetime(clean_df['date'], utc=True)
clean_df.dtypes

C:\Users\anton\AppData\Local\Temp\ipykernel_35496\1401035101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['date'] = pd.to_datetime(clean_df['date'], utc=True)


date         datetime64[ns, UTC]
Symbol                    object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Volume                     int64
Dividends                float64
dtype: object

### make gap attribute

In [50]:
def gap_percentage(open_price, prev_close): return (open_price - prev_close) / prev_close * 100
gap_threshold = 5

grouped_data = clean_df.groupby('Symbol')
grouped_data = grouped_data.apply(lambda x: gap_percentage(x['Open'], x['Close'].shift(1))).reset_index()
gap_data = grouped_data[(grouped_data['gap'] > gap_threshold) | (grouped_data['gap'] < -gap_threshold)]
gap_data = gap_data.reset_index()
gap_data = pd.merge(gap_data, clean_df, on=['Symbol', 'date'])
gap_data


C:\Users\anton\AppData\Local\Temp\ipykernel_35496\1912334985.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['gap'] = None


KeyError: 'gap'